In [2]:
# Import necessary libraries
import weaviate
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Weaviate
from langchain.embeddings import HuggingFaceEmbeddings

In [4]:
# Set API Keys and Weaviate Cluster details
WEAVIATE_API_KEY = ""  # Replace with your actual API key
WEAVIATE_CLUSTER = ""  # Replace with your Weaviate cluster URL

In [19]:
import os

# Check if the environment variables are set correctly
# print("WEAVIATE_URL:", os.environ.get(WEAVIATE_URL))
print("WEAVIATE_API_KEY:", os.environ.get(WEAVIATE_API_KEY))


WEAVIATE_API_KEY: None


In [13]:
import os
import weaviate
from weaviate.auth import AuthApiKey

# Best practice: store your credentials in environment variables
wcd_url = os.environ[""]  # Use the key "WEAVIATE_URL"
wcd_api_key = os.environ[""]  # Use the key "WEAVIATE_API_KEY"

# Initialize the Weaviate client using the correct method
client = weaviate.Client(
    url=wcd_url,  # Your Weaviate Cloud URL
    auth_client_secret=AuthApiKey(wcd_api_key)  # API Key for authentication
)

# Check if the client is ready
print("Weaviate connected:", client.is_ready())  # Method to check connection status


KeyError: 'https://o5jj81kzsl6gjvxyq2m5vq.c0.us-east1.gcp.weaviate.cloud'

In [5]:
# Connect to Weaviate Cluster
auth_config = weaviate.auth.AuthApiKey(api_key=WEAVIATE_API_KEY)
client = weaviate.Client(
    url=WEAVIATE_CLUSTER,
    auth_client_secret=auth_config,
    startup_period=10
)

C:\Users\emon1\AppData\Local\Temp\ipykernel_10272\2895696113.py:3: DeprecationWarning: 
Python client v3 `weaviate.Client(...)` has been removed.

Upgrade your code to use Python client v4 `weaviate.WeaviateClient` connections and methods.
    - For Python Client v4 usage, see: https://weaviate.io/developers/weaviate/client-libraries/python
    - For code migration, see: https://weaviate.io/developers/weaviate/client-libraries/python/v3_v4_migration

If you have to use v3 code, install the v3 client and pin the v3 dependency in your requirements file: `weaviate-client>=3.26.7;<4.0.0`
  client = weaviate.Client(


TypeError: __init__() got an unexpected keyword argument 'url'

In [ ]:
# Connect to Weaviate Cluster
auth_config = weaviate.auth.AuthApiKey(api_key=WEAVIATE_API_KEY)
client = weaviate.Client(
    url=WEAVIATE_CLUSTER,
    auth_client_secret=auth_config,
    startup_period=10
)

# Check if Weaviate is ready
print("Weaviate is ready:", client.is_ready())

# Step 1: Load PDF Data
loader = DirectoryLoader("./data", glob="**/*.pdf")  # Loads all PDFs from the "data" folder
data = loader.load()
print(f"Loaded {len(data)} documents.")

# Step 2: Split Documents into Smaller Chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
docs = text_splitter.split_documents(data)
print(f"Split into {len(docs)} chunks.")

# Step 3: Initialize Hugging Face Embeddings (Alternative to OpenAI)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Step 4: Define Weaviate Schema (Database Structure)
client.schema.delete_all()  # Delete existing schema (if needed)
schema = {
    "classes": [
        {
            "class": "Chatbot",
            "description": "Documents for chatbot",
            "vectorizer": "text2vec-transformers",  # Using Hugging Face instead of OpenAI
            "moduleConfig": {"text2vec-transformers": {}},  # No extra config needed
            "properties": [
                {
                    "dataType": ["text"],
                    "description": "The content of the paragraph",
                    "name": "content",
                },
            ],
        },
    ]
}

client.schema.create(schema)
print("Weaviate schema created.")

# Step 5: Store Text Chunks in Weaviate
vectorstore = Weaviate(client, "Chatbot", "content", attributes=["source"])
text_meta_pair = [(doc.page_content, doc.metadata) for doc in docs]
texts, meta = list(zip(*text_meta_pair))
vectorstore.add_texts(texts, meta)
print("Documents uploaded to Weaviate.")

# Step 6: Perform Similarity Search
query = "What is YOLO?"
docs = vectorstore.similarity_search(query, top_k=5)  # Retrieve top 5 most relevant texts
print("Search results:", docs)

# Print retrieved documents
for i, doc in enumerate(docs, 1):
    print(f"\nResult {i}:\n", doc.page_content)


In [ ]:
# Install required libraries
!pip install weaviate-client langchain unstructured "unstructured[pdf]" sentence-transformers

# Import necessary libraries
import weaviate
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Weaviate
from langchain.embeddings import HuggingFaceEmbeddings

# Set API Keys and Weaviate Cluster details
WEAVIATE_API_KEY = "your-weaviate-api-key"  # Replace with your actual API key
WEAVIATE_CLUSTER = "https://your-cluster-url.weaviate.network"  # Replace with your Weaviate cluster URL

# Connect to Weaviate Cluster
auth_config = weaviate.auth.AuthApiKey(api_key=WEAVIATE_API_KEY)
client = weaviate.Client(
    url=WEAVIATE_CLUSTER,
    auth_client_secret=auth_config,
    startup_period=10
)

# Check if Weaviate is ready
print("Weaviate is ready:", client.is_ready())

# Step 1: Load PDF Data
loader = DirectoryLoader("./data", glob="**/*.pdf")  # Loads all PDFs from the "data" folder
data = loader.load()
print(f"Loaded {len(data)} documents.")

# Step 2: Split Documents into Smaller Chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
docs = text_splitter.split_documents(data)
print(f"Split into {len(docs)} chunks.")

# Step 3: Initialize Hugging Face Embeddings (Alternative to OpenAI)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Step 4: Define Weaviate Schema (Database Structure)
client.schema.delete_all()  # Delete existing schema (if needed)
schema = {
    "classes": [
        {
            "class": "Chatbot",
            "description": "Documents for chatbot",
            "vectorizer": "text2vec-transformers",  # Using Hugging Face instead of OpenAI
            "moduleConfig": {"text2vec-transformers": {}},  # No extra config needed
            "properties": [
                {
                    "dataType": ["text"],
                    "description": "The content of the paragraph",
                    "name": "content",
                },
            ],
        },
    ]
}

client.schema.create(schema)
print("Weaviate schema created.")

# Step 5: Store Text Chunks in Weaviate
vectorstore = Weaviate(client, "Chatbot", "content", attributes=["source"])
text_meta_pair = [(doc.page_content, doc.metadata) for doc in docs]
texts, meta = list(zip(*text_meta_pair))
vectorstore.add_texts(texts, meta)
print("Documents uploaded to Weaviate.")

# Step 6: Perform Similarity Search
query = "What is YOLO?"
docs = vectorstore.similarity_search(query, top_k=5)  # Retrieve top 5 most relevant texts
print("Search results:", docs)

# Print retrieved documents
for i, doc in enumerate(docs, 1):
    print(f"\nResult {i}:\n", doc.page_content)
